In [1]:
#현재 수준에서 진행이 안되는 것으로 확인, diff(닭고기 소고기 쇠고기 구분 못함)나 kolnpy 수준에서 단순 대조만 가능 

In [2]:
import pandas as pd
import re
df_allergy = pd.read_csv('data/0515.csv')
df_allergy

,Unnamed: 0,rnum,rawmtrl,allergy
0,0,1,"찹쌀,김,참깨,옥수수기름(옥배유),양파,무,대파,천일염,마늘,새우,멸치,다시마,건표...",없음
1,1,2,"찹쌀,김,참깨,옥수수기름(옥배유),아몬드,양파,무,천일염,대파,마늘,새우,멸치,다시...",아몬드
2,2,3,"찹쌀,김,참깨,옥수수기름(옥배유),양파,무,대파,천일염,마늘,새우,멸치,다시마,건표...",없음
3,3,4,군고구마,없음
4,4,5,군고구마,없음
...,...,...,...,...
15087,15087,15088,"된장63.15%[대두{수입산(미국산,중국산,캐나다산)},소맥분(밀:미국산,호주산),...","대두,밀"
15088,15088,15089,"물엿,소맥분(밀:미국산,호주산),혼합양념(중국산)[고춧가루6.12%,(고추:중국산)...",밀
15089,15089,15090,"탈지대두[수입산(인도산,미국산,중국산)],정제수,양조간장원액(대만산)[탈지대두,소맥...","대두,밀"
15090,15090,15091,"정제수,아미노산액[탈지대두:외국산(인도산,미국산,중국산등)],정제소금(국산),액상과...","대두,밀"


In [3]:
from soynlp.normalizer import *
import re

def preprocessing(text):
    # 문제를 일으킬 수 있는 문자 제거
    bad_chars = {"\u200b": "", "…": " ... ", "\ufeff": ""}
    for bad_char in bad_chars:
        text = text.replace(bad_char, bad_chars[bad_char])
        
    error_chars = {"\u3000": " ", "\u2009": " ", "\u2002": " ", "\xa0":" "}
    for error_char in error_chars:
        text = text.replace(error_char, error_chars[error_char])
    
    # ~산 제거 (아황산 제외)
    text = re.sub(r'\b(?!아황산)\w+산\b', '', text).strip()

    # 이메일 제거
    text = re.sub(r"[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+", "[이메일]", text).strip()
    
    # "#문자" 형식 어절 제거
    text = re.sub(r"#\S+", "", text).strip()
    
    # "@문자" 형식 어절 제거
    text = re.sub(r"@\w+", "", text).strip()
    
    # URL 제거
    text = re.sub(r"(http|https)?:\/\/\S+\b|www\.(\w+\.)+\S*", "[웹주소]", text).strip()
    text = re.sub(r"pic\.(\w+\.)+\S*", "[웹주소]", text).strip()
    
    # 뉴스 저작권 관련 텍스트 제거
    re_patterns = [
        r"\<저작권자(\(c\)|ⓒ|©|\(Copyright\)|(\(c\))|(\(C\))).+?\>",
        r"저작권자\(c\)|ⓒ|©|(Copyright)|(\(c\))|(\(C\))"
    ]
    
    for re_pattern in re_patterns:
        text = re.sub(re_pattern, "", text).strip()
    
    # 뉴스 내 포함된 이미지에 대한 레이블 제거
    text = re.sub(r"\(출처 ?= ?.+\) |\(사진 ?= ?.+\) |\(자료 ?= ?.+\)| \(자료사진\) |사진=.+기자 ", "", text).strip()
    
    # 중복 문자 처리
    text = repeat_normalize(text, num_repeats=2).strip()
    
    # 문제를 일으킬 수 있는 구두점 치환
    punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', }
    for p in punct_mapping:
        text = text.replace(p, punct_mapping[p])
    
    # 연속된 공백 치환
    text = re.sub(r"\s+", " ", text).strip()
    
    # 개행 문자 "\n" 제거
    text = text.replace('\n', '')
    
    # 영어 제거
    text = re.sub(r'[a-zA-Z]', '', text)
    return text

In [4]:
# def remove_useless_breacket_l(texts):
#     """
#     위키피디아 전처리를 위한 함수입니다.
#     괄호 내부에 의미가 없는 정보를 제거합니다.
#     아무런 정보를 포함하고 있지 않다면, 괄호를 통채로 제거합니다.
#     ``수학(,)`` -> ``수학``
#     ``수학(數學,) -> ``수학(數學)``
#     """
#     bracket_pattern = re.compile(r"\((.*?)\)")
#     preprocessed_text = []
#     for text in texts:
#         modi_text = ""
#         text = text.replace("()", "")  # 수학() -> 수학
#         brackets = bracket_pattern.search(text)
#         if not brackets:
#             if text:
#                 preprocessed_text.append(text)
#                 continue
#         replace_brackets = {}
#         # key: 원본 문장에서 고쳐야하는 index, value: 고쳐져야 하는 값
#         # e.g. {'2,8': '(數學)','34,37': ''}
#         while brackets:
#             index_key = str(brackets.start()) + "," + str(brackets.end())
#             bracket = text[brackets.start() + 1 : brackets.end() - 1]
#             infos = bracket.split(",")
#             modi_infos = []
#             for info in infos:
#                 info = info.strip()
#                 if len(info) > 0:
#                     modi_infos.append(info)
#             if len(modi_infos) > 0:
#                 replace_brackets[index_key] = "(" + ", ".join(modi_infos) + ")"
#             else:
#                 replace_brackets[index_key] = ""
#             brackets = bracket_pattern.search(text, brackets.start() + 1)
#         end_index = 0
#         for index_key in replace_brackets.keys():
#             start_index = int(index_key.split(",")[0])
#             modi_text += text[end_index:start_index]
#             modi_text += replace_brackets[index_key]
#             end_index = int(index_key.split(",")[1])
#         modi_text += text[end_index:]
#         modi_text = modi_text.strip()
#         if modi_text:
#             preprocessed_text.append(modi_text)
#     return preprocessed_text


In [5]:
import re

def remove_useless_breacket(text):
    """
    위키피디아 전처리를 위한 함수입니다.
    괄호 내부에 의미가 없는 정보를 제거합니다.
    아무런 정보를 포함하고 있지 않다면, 괄호를 통채로 제거합니다.
    ``수학(,)`` -> ``수학``
    ``수학(數學,) -> ``수학(數學)``
    """
    bracket_pattern = re.compile(r"\((.*?)\)|\[(.*?)\]|\{(.*?)\}")
    modi_text = ""
    text = text.replace("()", "")  # 수학() -> 수학
    brackets = bracket_pattern.search(text)
    if not brackets:
        if text:
            return text
    replace_brackets = {}
    # key: 원본 문장에서 고쳐야하는 index, value: 고쳐져야 하는 값
    # e.g. {'2,8': '(數學)','34,37': ''}
    while brackets:
        index_key = str(brackets.start()) + "," + str(brackets.end())
        bracket = brackets.group(0)[1:-1]
        infos = bracket.split(",")
        modi_infos = []
        for info in infos:
            info = info.strip()
            if len(info) > 0:
                modi_infos.append(info)
        if len(modi_infos) > 0:
            replace_brackets[index_key] = "(" + ", ".join(modi_infos) + ")"
        else:
            replace_brackets[index_key] = ""
        brackets = bracket_pattern.search(text, brackets.start() + 1)
    end_index = 0
    for index_key in replace_brackets.keys():
        start_index = int(index_key.split(",")[0])
        modi_text += text[end_index:start_index]
        if replace_brackets[index_key] and (not modi_text or modi_text[-1] != ","):
            modi_text += ","
        modi_text += replace_brackets[index_key]
        end_index = int(index_key.split(",")[1])
    modi_text += text[end_index:]
    modi_text = modi_text.strip()
    return modi_text

# 텍스트 스트링을 함수에 적용하여 수정
text = "파이썬(,)은(는) [데이터 분석]{data analysis} {머신 러닝}(machine learning) {} [] 프로그래밍 언어입니다."
modified_text = remove_useless_breacket(text)
print(modified_text)


파이썬은,(는) ,(데이터 분석),(data analysis) ,(머신 러닝),(machine learning)   프로그래밍 언어입니다.


In [6]:
def remove_bad_char(texts):
    bad_chars = {"\u200b": "", "…": " ... ", "\ufeff": ""}
    preprocessed_text  = []
    for text in texts:
        for bad_char in bad_chars:
            text = text.replace(bad_char, bad_chars[bad_char])
        text = re.sub(r"[+á?\xc3\xa1]", "", text)
        if text:
            preprocessed_text.append(text)
    return preprocessed_text

In [7]:
def remove_email(texts):
    preprocessed_text  = []
    for text in texts:
        text = re.sub(r"[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+", "", text).strip()
        if text:
            preprocessed_text.append(text)
    return preprocessed_text

In [8]:
def remove_numbers(text):
    # 정규표현식을 사용하여 숫자를 제거합니다
    cleaned_text = re.sub(r'\d+', '', text)
    return cleaned_text

In [9]:
def remove_hashtag(texts):
    preprocessed_text  = []
    for text in texts:
        text = re.sub(r"#\S+", "", text).strip()
        if text:
            preprocessed_text.append(text)
    return preprocessed_text

In [10]:
def clean_punc(texts):
    punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', }
    preprocessed_text  = []
    for text in texts:
        for p in punct_mapping:
            text = text.replace(p, punct_mapping[p])
        text = text.strip()
        if text:
            preprocessed_text.append(text)
    return preprocessed_text

In [11]:
def clean_text(text):
    # 문자열에서 %, /, \, :를 제거합니다
    cleaned_text = text.replace('<', '').replace('>', '').replace('【', '').replace('】', '').replace('^', '').replace('%', '').replace('/', '').replace('\\', '').replace(':', '').replace(';', '').replace('*', '').replace('-', '').replace('등급', '').replace('이상', '').replace('미국', '').replace('호주', '').replace('베트남', '').replace('프랑스', '').replace('독일', '').replace('아르헨티나', '').replace('일본', '').replace('중국', '').replace('캐나다', '').replace('러시아', '').replace('말레이시아', '').replace('칠레', '').replace('브라질', '').replace('인도네시아', '').replace('인도', '').replace('담양', '').replace('소고기', '쇠고기').replace('오스트리아', '').replace('터키', '').replace('콜롬비아', '').replace('º', '').replace('세균수기준', '').replace('우크라이나',"").replace("싱가포르","").replace("네덜란드","")
    # 문자열에서 .을 ,로 바꿉니다
    cleaned_text = cleaned_text.replace('.', ',')
    return cleaned_text

In [12]:
def add_spaces(text):
    # 정규표현식을 사용하여 특수문자 앞뒤에 공백을 추가합니다
    cleaned_text = re.sub(r'([,.\(\)\[\]{}/\\:;%*])', r' \1 ', text)
    return cleaned_text

In [13]:
import re

def replace_brackets(text):
    # 괄호 및 문장 내 공백을 , 로 변경
    text = re.sub(r"[()\[\]{}]", ",", text)
    # 문장 내 공백 제거
    text = text.replace(" ", "")
    # 중복된 , 를 1개로 변경
    text = re.sub(r",+", ", ", text)
        
    return text

# 텍스트 스트링을 함수에 적용하여 수정
text = "파이썬(,)은(는) [데이터 분석]{data analysis} {머신 러닝}(machine learning) 프로그래밍 언어입니다."
modified_text = replace_brackets(text)
print(modified_text)

파이썬, 은, 는, 데이터분석, dataanalysis, 머신러닝, machinelearning, 프로그래밍언어입니다.


In [14]:
def remove_duplicates(text):
    # 문장을 단어로 분할하여 리스트로 저장
    words = text.split(", ")
    
    # 중복된 단어를 제거하여 새로운 리스트 생성
    unique_words = list(set(words))
    
    # 단어들을 다시 ', '로 구분하여 하나의 문자열로 반환
    modified_text = ", ".join(unique_words)

    # 문장의 제일 앞에 있는 ', '를 제거
    modified_text = modified_text.lstrip(", ")
    
    return modified_text

# 텍스트 스트링을 함수에 적용하여 수정
text = "apple, banana, cherry, apple, durian, cherry"
modified_text = remove_duplicates(text)
print(modified_text)

cherry, banana, apple, durian


In [15]:
import re

def remove_words_with_suffix(text, suffix):
    pattern = r'\b\w+' + suffix + r'\b'
    result = re.sub(pattern, '', text)
    return result.strip()

# 예시 텍스트
text = "사과, 배, 딸기, 바나나, 오렌지, 수박등, 복숭아등, 포도, 체리등"

# '등'으로 끝나는 단어 제거 적용
modified_text = remove_words_with_suffix(text, "등")

print(modified_text)

사과, 배, 딸기, 바나나, 오렌지, , , 포도,


In [16]:

def remove_pattern_words(text):
    patterns = ['국가', '홈페이지', '별도', '표기','※','특정','성분','성분명','및','함량']
    result = []
    
    words = text.split(',')
    for word in words:
        word = word.strip()  # 단어 앞뒤 공백 제거
        
        # 패턴 단어 포함 여부 확인
        if not any(pattern in word for pattern in patterns):
            result.append(word)
    
    result_text = ', '.join(result)
    return result_text

In [17]:
#df_allergy['rawmtrl_list'] = 
df_allergy['rawmtrl'][0]

'찹쌀,김,참깨,옥수수기름(옥배유),양파,무,대파,천일염,마늘,새우,멸치,다시마,건표고버섯,둥굴레,감초,정제수'

In [18]:
df_allergy['preprocessing'] = df_allergy['rawmtrl'].apply(add_spaces).apply(remove_numbers).apply(remove_pattern_words).apply(preprocessing).apply(clean_text).apply(lambda x: remove_words_with_suffix(x, '등')).apply(remove_useless_breacket).apply(replace_brackets).apply(remove_duplicates).apply(replace_brackets)


In [23]:
df_allergy.sample(10) #12547
#df_allergy['preprocessing'][13907]#

,Unnamed: 0,rnum,rawmtrl,allergy,preprocessing
12781,12781,12782,"고추장85.6%{물엿, 고과당, 소맥분(밀:미국, 호주산), 고추양념(중국산), 쌀...",알수없음,"설탕, 쌀, 고추장, 매실농축액, 물엿, 고추양념, 소맥분, 고추장용콩메주, 마늘,..."
10047,10047,10048,"정제수, 맥심콜드브루 커피추출액(고형분 2.0%이상, 커피원두 : 콜롬비아산 40%...",알수없음,"비타민, 고형분, 탄산칼륨, 커피원두, 자당지방산에스테르, 맥심콜드브루커피추출액, 정제수"
8992,8992,8993,"돼지고기(국내산)59.61%,닭고기(국내산)11.45%,정제수,양파,기타설탕,대파,...",알수없음,"닭고기, 피로인산나트륨, 아로맥스, 스모크오일, 발효조미료, 참기름, 돼지고기, 정..."
8007,8007,8008,"현미(국산)60%,녹차(국산)40%",알수없음,"녹차, 현미"
14976,14976,14977,"말티톨시럽76.8%,자일리톨20.6%, D-소르비톨1%,혼합제제(젖산,사과산,구연산...",복숭아,"고형분, 자일리톨, 쿨리향, 글리세린지방산에스테르, 말티톨시럽, 자주색고구마색소, ..."
11042,11042,11043,"밀가루(밀:미국산), L-글루타민산나트륨(향미증진제), 정제소금(국산), 분리대두단...","밀, 대두","올레오레진캡시컴, 정제소금, 향미증진제, 흑후추분말, 매운맛베이스, 밀가루, 구아검..."
13221,13221,13222,"대두(국산100%), 1급해수(제조용제), 염화칼슘",대두,"제조용제, 대두, 염화칼슘, 급해수"
12520,12520,12521,"밀가루1(밀,미국산),혼합식용유(팜올레인유 말레이시아산,팜유 말레이시아산,올레오레진...",알수없음,"혼합식용유, 전지분유, 양파엑기스, 우유, 경화유, 정제소금, 생이스트, 산도조절제..."
5868,5868,5869,"흑삼추출액 100%(고형분 1% 이상, 국산)",알수없음,"고형분, 흑삼추출액"
12060,12060,12061,"면/소맥분(호주산,미국산),변성전분,팜유(말레이시아산),정제염(국내산),미감에스유,...",알수없음,"변성전분, 사과농축액, 태양초고추장, 건양배추, 구아검, 정제염, 참기름, 양조식초..."


s